<a href="https://colab.research.google.com/github/domzegar/colab_notebooks/blob/main/AutoEncoders%20-%20GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#AutoEncoders

##Downloading the dataset

###ML-100K

In [1]:
!wget "http://files.grouplens.org/datasets/movielens/ml-100k.zip"
!unzip ml-100k.zip
!ls

--2022-11-06 09:38:44--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

ml-100k.zip         100%[===================>]   4.70M  2.69MB/s    in 1.7s    

2022-11-06 09:38:47 (2.69 MB/s) - ‘ml-100k.zip’ saved [4924029/4924029]

Archive:  ml-100k.zip
   creating: ml-100k/
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base

###ML-1M

In [2]:
!wget "http://files.grouplens.org/datasets/movielens/ml-1m.zip"
!unzip ml-1m.zip
!ls

--2022-11-06 09:50:33--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip’

ml-1m.zip           100%[===================>]   5.64M  3.33MB/s    in 1.7s    

2022-11-06 09:50:36 (3.33 MB/s) - ‘ml-1m.zip’ saved [5917549/5917549]

Archive:  ml-1m.zip
   creating: ml-1m/
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         
ml-100k  ml-100k.zip  ml-1m  ml-1m.zip	sample_data


##Importing the libraries

In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

## Importing the dataset


In [4]:
# We won't be using this dataset.
movies = pd.read_csv('ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

## Preparing the training set and the test set


In [5]:
training_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t')
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('ml-100k/u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')

## Getting the number of users and movies


In [6]:
nb_users = int(max(max(training_set[:, 0], ), max(test_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1], ), max(test_set[:, 1])))

## Converting the data into an array with users in lines and movies in columns


In [7]:
def convert(data):
  new_data = []
  for id_users in range(1, nb_users + 1):
    id_movies = data[:, 1] [data[:, 0] == id_users]
    id_ratings = data[:, 2] [data[:, 0] == id_users]
    ratings = np.zeros(nb_movies)
    ratings[id_movies - 1] = id_ratings
    new_data.append(list(ratings))
  return new_data
training_set = convert(training_set)
test_set = convert(test_set)

In [11]:
len(training_set[0])

1682

In [12]:
movies.shape

(3883, 3)

## Converting the data into Torch tensors


In [13]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [15]:
class SAE(nn.Module):
  def __init__(self, ):
    super(SAE, self).__init__()
    self.fc1 = nn.Linear(nb_movies, 20)
    self.fc2 = nn.Linear(20, 10)
    self.fc3 = nn.Linear(10, 20)
    self.fc4 = nn.Linear(20, nb_movies)
    self.activation = nn.Sigmoid()

  def forward(self, x):
    x = self.activation(self.fc1(x))
    x = self.activation(self.fc2(x))
    x = self.activation(self.fc3(x))
    x = self.fc4(x)
    return x

In [18]:
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr=0.05, weight_decay=0.5)

In [21]:
for param in sae.parameters():
  print(param.shape)

torch.Size([20, 1682])
torch.Size([20])
torch.Size([10, 20])
torch.Size([10])
torch.Size([20, 10])
torch.Size([20])
torch.Size([1682, 20])
torch.Size([1682])


In [28]:
nb_epoch = 200
for epoch in range(1, nb_epoch+1):
  train_loss = 0
  s = 0.
  for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = input.clone()
    if torch.sum(target.data > 0) > 0:
      output = sae.forward(input)
      target.require_grad = False
      output[target == 0] = 0
      loss = criterion(output, target)
      mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
      loss.backward()
      train_loss += np.sqrt(loss.item()*mean_corrector)
      s += 1.
      optimizer.step()
  print(f"epoch: {epoch} loss: {train_loss/s}")

epoch: 1 loss: 1.819105591267047
epoch: 2 loss: 1.1743735457326783
epoch: 3 loss: 1.1719071113991628
epoch: 4 loss: 1.1311741011209102
epoch: 5 loss: 1.152095616482916
epoch: 6 loss: 1.1206091666782443
epoch: 7 loss: 1.1433776590252047
epoch: 8 loss: 1.1156168324960714
epoch: 9 loss: 1.1393176358954606
epoch: 10 loss: 1.1159119226467245
epoch: 11 loss: 1.1350406794252985
epoch: 12 loss: 1.1119173084377283
epoch: 13 loss: 1.1321008878205827
epoch: 14 loss: 1.1076472279922833
epoch: 15 loss: 1.1294564532797156
epoch: 16 loss: 1.1014261067810573
epoch: 17 loss: 1.121797918402122
epoch: 18 loss: 1.0980830674974509
epoch: 19 loss: 1.119060550364285
epoch: 20 loss: 1.097184437351998
epoch: 21 loss: 1.117400457603144
epoch: 22 loss: 1.0956256233155135
epoch: 23 loss: 1.112086280636907
epoch: 24 loss: 1.0875000133196047
epoch: 25 loss: 1.1042584524700347
epoch: 26 loss: 1.080747053530965
epoch: 27 loss: 1.0989716719232776
epoch: 28 loss: 1.0785711165751692
epoch: 29 loss: 1.0980872668109856
ep

In [35]:
torch.cuda.is_available()

True

In [34]:
# Calculating test loss
test_loss = 0
s = 0.
for id_user in range(nb_users):
  input = Variable(training_set[id_user]).unsqueeze(0)
  target = Variable(test_set[id_user]).unsqueeze(0)
  if torch.sum(target.data > 0) > 0:
    output = sae(input)
    target.require_grad = False
    output[target == 0] = 0
    loss = criterion(output, target)
    mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
    test_loss += np.sqrt(loss.item()*mean_corrector)
    s += 1.
print(f"Test loss: {test_loss/s} number of users: {s}")

Test loss: 0.9818580483738666 number of users: 459.0


In [ ]:
nb_epoch = 200
for epoch in range(1, nb_epoch+1):
  train_loss = 0
  s = 0.
  for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = input.clone()
    if torch.sum(target.data > 0) > 0:
      output = sae.forward(input)
      target.require_grad = False
      output[target == 0] = 0
      loss = criterion(output, target)
      mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
      loss.backward()
      train_loss += np.sqrt(loss.item()*mean_corrector)
      s += 1.
  print(f"epoch: {epoch} loss: {train_loss/s}")

## Creating the architecture of the Neural Network


In [ ]:
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)

## Training the SAE


In [ ]:
nb_epoch = 200
for epoch in range(1, nb_epoch + 1):
  train_loss = 0
  s = 0.
  for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = input.clone()
    if torch.sum(target.data > 0) > 0:
      output = sae(input)
      target.require_grad = False
      output[target == 0] = 0
      loss = criterion(output, target)
      mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
      loss.backward()
      train_loss += np.sqrt(loss.data*mean_corrector)
      s += 1.
      optimizer.step()
  print('epoch: '+str(epoch)+'loss: '+ str(train_loss/s))

epoch: 1loss: tensor(1.7709)
epoch: 2loss: tensor(1.0968)
epoch: 3loss: tensor(1.0534)
epoch: 4loss: tensor(1.0384)
epoch: 5loss: tensor(1.0309)
epoch: 6loss: tensor(1.0267)
epoch: 7loss: tensor(1.0239)
epoch: 8loss: tensor(1.0220)
epoch: 9loss: tensor(1.0208)
epoch: 10loss: tensor(1.0198)
epoch: 11loss: tensor(1.0187)
epoch: 12loss: tensor(1.0185)
epoch: 13loss: tensor(1.0179)
epoch: 14loss: tensor(1.0173)
epoch: 15loss: tensor(1.0171)
epoch: 16loss: tensor(1.0170)
epoch: 17loss: tensor(1.0165)
epoch: 18loss: tensor(1.0168)
epoch: 19loss: tensor(1.0163)
epoch: 20loss: tensor(1.0163)
epoch: 21loss: tensor(1.0162)
epoch: 22loss: tensor(1.0161)
epoch: 23loss: tensor(1.0158)
epoch: 24loss: tensor(1.0160)
epoch: 25loss: tensor(1.0153)
epoch: 26loss: tensor(1.0156)
epoch: 27loss: tensor(1.0154)
epoch: 28loss: tensor(1.0151)
epoch: 29loss: tensor(1.0141)
epoch: 30loss: tensor(1.0115)
epoch: 31loss: tensor(1.0098)
epoch: 32loss: tensor(1.0075)
epoch: 33loss: tensor(1.0052)
epoch: 34loss: tens

## Testing the SAE


In [ ]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
  input = Variable(training_set[id_user]).unsqueeze(0)
  target = Variable(test_set[id_user]).unsqueeze(0)
  if torch.sum(target.data > 0) > 0:
    output = sae(input)
    target.require_grad = False
    output[target == 0] = 0
    loss = criterion(output, target)
    mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
    test_loss += np.sqrt(loss.data*mean_corrector)
    s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(0.9681)
